In [4]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, ForeignKey, select

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
base = declarative_base()

In [7]:
engine = create_engine("sqlite:///newalchemy.db", echo = True)

In [36]:
class User(base):
    __tablename__ = 'T_USER'
    
    pk = Column('C_PK',Integer,primary_key=True)
    name = Column('C_NAME',String)
    
    def __repr__(self):
        return '{}{}'.format(self.pk,self.name)

In [9]:
base.metadata.create_all(engine)

2021-07-06 10:22:54,093 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-06 10:22:54,094 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2021-07-06 10:22:54,094 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-06 10:22:54,096 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2021-07-06 10:22:54,096 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-06 10:22:54,097 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"C_PK" INTEGER NOT NULL, 
	"C_NAME" VARCHAR, 
	PRIMARY KEY ("C_PK")
)


2021-07-06 10:22:54,098 INFO sqlalchemy.engine.Engine [no key 0.00056s] ()
2021-07-06 10:22:54,099 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
User.__table__
#생성할 테이블에 대한 정보

Table('T_USER', MetaData(), Column('C_PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('C_NAME', String(), table=<T_USER>), schema=None)

In [12]:
User.__mapper__

<Mapper at 0x7fcddf0310d0; User>

In [13]:
kim = User(name = 'kim')

In [15]:
kim.name, kim.pk

('kim', None)

In [16]:
#인스턴스가 변경이 되면 싱크가 깨지니깐 디비를 업데이트 해야한다. 커밋을 해야한다.

In [17]:
from sqlalchemy.orm import sessionmaker

In [18]:
from sqlalchemy.orm.session import Session

In [19]:
sess = sessionmaker()
session = sess(bind = engine)

In [20]:
session.add(kim)

In [21]:
session.dirty

IdentitySet([])

In [22]:
session.is_modified(kim)

True

In [24]:
session.commit()

2021-07-06 10:31:59,709 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-06 10:31:59,710 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("C_NAME") VALUES (?)
2021-07-06 10:31:59,711 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('kim',)
2021-07-06 10:31:59,712 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
kim.pk

2021-07-06 10:32:56,575 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-06 10:32:56,578 INFO sqlalchemy.engine.Engine SELECT "T_USER"."C_PK" AS "T_USER_C_PK", "T_USER"."C_NAME" AS "T_USER_C_NAME" 
FROM "T_USER" 
WHERE "T_USER"."C_PK" = ?
2021-07-06 10:32:56,578 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (1,)


1

In [26]:
kim.pk, kim.name

(1, 'kim')

In [27]:
kim.name = 'dfasfaslfjweqnfo'

In [29]:
session.commit()

2021-07-06 10:34:59,740 INFO sqlalchemy.engine.Engine UPDATE "T_USER" SET "C_NAME"=? WHERE "T_USER"."C_PK" = ?
2021-07-06 10:34:59,741 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ('dfasfaslfjweqnfo', 1)
2021-07-06 10:34:59,742 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
#매핑한 데이터가 싱크가 안맞으니 커밋으로 푸쉬를 해야 디비상의 변화가 생긴다.

In [31]:
kim.name

2021-07-06 10:35:22,808 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-06 10:35:22,809 INFO sqlalchemy.engine.Engine SELECT "T_USER"."C_PK" AS "T_USER_C_PK", "T_USER"."C_NAME" AS "T_USER_C_NAME" 
FROM "T_USER" 
WHERE "T_USER"."C_PK" = ?
2021-07-06 10:35:22,809 INFO sqlalchemy.engine.Engine [cached since 146.2s ago] (1,)


'dfasfaslfjweqnfo'

--------

In [38]:
kim.name = 'TEST'

In [42]:
session.commit()

2021-07-06 10:43:09,832 INFO sqlalchemy.engine.Engine UPDATE "T_USER" SET "C_NAME"=? WHERE "T_USER"."C_PK" = ?
2021-07-06 10:43:09,833 INFO sqlalchemy.engine.Engine [cached since 490.1s ago] ('TEST', 1)
2021-07-06 10:43:09,834 INFO sqlalchemy.engine.Engine COMMIT


In [43]:
session.dirty

IdentitySet([])

In [48]:
user_list = list()
for _ in list(session.query(User)):
    user_list.append(_)
    print(type(_))

2021-07-06 10:46:36,404 INFO sqlalchemy.engine.Engine SELECT "T_USER"."C_PK" AS "T_USER_C_PK", "T_USER"."C_NAME" AS "T_USER_C_NAME" 
FROM "T_USER"
2021-07-06 10:46:36,405 INFO sqlalchemy.engine.Engine [cached since 79.96s ago] ()
<class '__main__.User'>


In [54]:
user_list[0]

1TEST

In [55]:
user_list[0].name='kim'

In [56]:
session.dirty

IdentitySet([1kim])

In [57]:
session.commit()

2021-07-06 10:47:44,452 INFO sqlalchemy.engine.Engine UPDATE "T_USER" SET "C_NAME"=? WHERE "T_USER"."C_PK" = ?
2021-07-06 10:47:44,453 INFO sqlalchemy.engine.Engine [generated in 0.00098s] ('kim', 1)
2021-07-06 10:47:44,454 INFO sqlalchemy.engine.Engine COMMIT


In [58]:
user_list[0]

2021-07-06 10:47:52,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-06 10:47:52,763 INFO sqlalchemy.engine.Engine SELECT "T_USER"."C_PK" AS "T_USER_C_PK", "T_USER"."C_NAME" AS "T_USER_C_NAME" 
FROM "T_USER" 
WHERE "T_USER"."C_PK" = ?
2021-07-06 10:47:52,764 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1,)


1kim

In [59]:
#세션에 더티가 있는지 없는지 보고, 커밋말 해주면 디비가 업데이트 된다!

------

In [74]:
class Artist(base):
    __tablename__ = 'T_ARTIST'
    __table_args__ = {'extend_existing': True}
    pk = Column('C_PK', Integer, primary_key = True)
    name = Column('C_NAME', String)
    
    def __repr__(self):
        return '{}{}'.format(self.pk, self.name)
        
        
class Album(base):
    __tablename__ = 'T_ALBUM'
    __table_args__ = {'extend_existing': True}
    pk = Column('C_PK', Integer, primary_key = True)
    name = Column('C_NAME', String)
    fk = Column('C_FK',None,ForeignKey('T_ARTIST.C_PK'))
    
    def __repr__(self):
        return '{}{}{}'.format(self.pk, self.name, self.fk)
    
class Genre(base):
    __tablename__ = 'T_GENRE'
    __table_args__ = {'extend_existing': True}
    pk = Column('C_PK', Integer, primary_key = True)
    name = Column('C_NAME', String)
    
    def __repr__(self):
        return '{}{}'.format(self.pk, self.name)
    
class Track(base):
    __tablename__ = 'T_TRACK'
    __table_args__ = {'extend_existing': True}
    pk = Column('C_PK', Integer, primary_key = True)
    name = Column('C_NAME', String)
    fk1 = Column('C_FK1',None,ForeignKey('T_ALBUM.C_PK'))
    fk2 = Column('C_FK2',None,ForeignKey('T_GENRE.C_PK'))

    
    def __repr__(self):
        return '{}{}{}{}'.format(self.pk, self.name, self.fk1, self.fk2)

In [73]:
base.metadata.clear()

In [75]:
artist1 = Artist(name = 'TOMCAAT')
artist2 = Artist(name = 'SPACEJIHUNNID')

In [86]:
session.add_all([artist1,artist2])

In [87]:
session.dirty

IdentitySet([])

In [88]:
session.is_modified(artist1)

True

In [79]:
session.commit()

2021-07-06 11:19:49,103 INFO sqlalchemy.engine.Engine COMMIT


In [92]:
album_list = [Album(name = '소년과시간여행의순기능',fk=artist1.pk),
             Album(name = 'EMERGENCY!',fk=artist2.pk)]

In [93]:
session.dirty

IdentitySet([])

In [94]:
session.add_all(album_list)

In [95]:
session.commit()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: T_ALBUM
[SQL: INSERT INTO "T_ALBUM" ("C_NAME", "C_FK") VALUES (?, ?)]
[parameters: ('소년과시간여행의순기능', None)]
(Background on this error at: http://sqlalche.me/e/14/e3q8) (Background on this error at: http://sqlalche.me/e/14/7s2a)